In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

headers = {'user-agent': 'Mozilla/5.0'}
list_todos = []

In [8]:
for page_number in range(1,17):

    resposta = requests.get(f"https://www.magazineluiza.com.br/busca/nintendo+switch/?page={page_number}",
                            headers=headers)

    sopa = resposta.text

    sopa_bonita = BeautifulSoup(sopa,'html.parser')

    list_titulo = sopa_bonita.find_all('h2',{'data-testid':'product-title'})

    list_preco_promo = sopa_bonita.find_all('p',{'data-testid':'price-value'})

    list_condition_promo = sopa_bonita.find_all('span',{'data-testid':'in-cash'})

    list_parcelamento = sopa_bonita.find_all('p',{'data-testid':'installment'})

    img_tags = sopa_bonita.find_all('img')
    img_srcs = [img['src'] for img in img_tags]

    for titulo, preco_promo,condition_promo, parcelado, img  in zip(list_titulo,list_preco_promo,list_condition_promo,list_parcelamento,img_srcs):
        titulo = titulo.text
        moeda = preco_promo.text
        moeda = moeda[0] + moeda[1]
        preco_promo = preco_promo.text.replace('R$','').replace('\xa0','').replace('.','').replace(',','.')
        condition_promo = condition_promo.text
        parcelado = parcelado.text.replace('R$','').replace('\xa0','').replace('.','').replace(',','.')
        imagem = str(img).replace('[', '').replace(']', '')

        list_todos.append((titulo,moeda,preco_promo,condition_promo,parcelado,imagem))

In [10]:
df = pd.DataFrame(list_todos, columns=['titulo','moeda','preco_promo','condition_promo','parcelado','imagem'])

In [11]:
df['preco_promo'] = df['preco_promo'].astype(np.float64).round(2)

In [12]:
def extrair_memoria(info):
        # Usar regex para encontrar padrões "GB" ou "Gb" ou "gb" seguidos por números
    padrao = r'(\d+)\s*(G[gBb])'
    resultado = re.search(padrao, info)
    if resultado:
        # Se encontrado, retornar a correspondência
        return resultado.group(0)
    else:
        # Se não encontrado, retornar uma string vazia
        return '-'

# Aplicar a função e criar a nova coluna "Memoria"
df['memoria'] = df['titulo'].apply(extrair_memoria)

In [14]:
oled = df['titulo'].str.contains('Oled', case=False)
oled = oled.replace({True: 'Sim', False: 'Nao'})
df['oled'] = oled

lite = df['titulo'].str.contains('Lite', case=False)
lite = lite.replace({True: 'Sim', False: 'Nao'})
df['lite'] = lite

controle = df['titulo'].str.contains('Joy-con', case=False)
controle = controle.replace({True: 'Sim', False: 'Nao'})
df['joy_con'] = controle

In [15]:
df

,titulo,moeda,preco_promo,condition_promo,parcelado,imagem,memoria,oled,lite,joy_con
0,Nintendo Switch Lite Turquesa 32GB - HDHSBAZA1BRA,R$,1142.10,no Pix,ou 1269.00 em 10x de 126.90 sem juros,https://a-static.mlcdn.com.br/280x210/nintendo...,32GB,Nao,Sim,Nao
1,Console Nintendo Switch Oled 64GB Cinza Edição...,R$,2279.50,no Pix,ou 2350.00 em 10x de 235.00 sem juros,https://a-static.mlcdn.com.br/280x210/console-...,64GB,Sim,Nao,Nao
2,Nintendo Switch 64GB 2 Controles Joy-con,R$,2564.05,no Pix,ou 2699.00 em 10x de 269.90 sem juros,https://i.mlcdn.com.br/selo-ml/65x50/e412ff6a-...,64GB,Nao,Nao,Sim
3,Console Nintendo Switch Mario Kart 8 Digital D...,R$,2085.50,no Pix,ou 2150.00 em 10x de 215.00 sem juros,https://a-static.mlcdn.com.br/280x210/nintendo...,-,Nao,Nao,Sim
4,"Nintendo Switch Lite 32GB Coral 5,5”",R$,1648.03,no Pix,ou 1699.00 em 10x de 169.90 sem juros,https://a-static.mlcdn.com.br/280x210/console-...,32GB,Nao,Sim,Nao
...,...,...,...,...,...,...,...,...,...,...
342,Adesivo Compatível Nintendo Switch Lite Skin -...,R$,44.99,no Pix,ou 186.89 em 4x de 46.72 sem juros,https://a-static.mlcdn.com.br/280x210/adesivo-...,-,Nao,Sim,Nao
343,Capa Case e Skin Adesivo Compatível Nintendo S...,R$,54.99,no Pix,ou 376.80 em 6x de 62.80 sem juros,https://a-static.mlcdn.com.br/280x210/capa-cas...,-,Nao,Nao,Nao
344,Sonic Mania - Switch,R$,160.48,no Pix,ou 376.80 em 6x de 62.80 sem juros,https://a-static.mlcdn.com.br/280x210/sonic-ma...,-,Nao,Nao,Nao
345,Dokapon Kingdom: Connect - Switch,R$,332.18,no Pix,ou 285.80 em 6x de 47.63 sem juros,https://a-static.mlcdn.com.br/280x210/dokapon-...,-,Nao,Nao,Nao
